# Machine learning for text classification
---

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
# importing a stylesheet library for visualisations from source[1] 
#plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')

In [2]:
#from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
#??CountVectorizer

In [3]:
dataframe = pd.read_csv("fake reviews dataset.csv")
dataframe.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [4]:
dataframe = dataframe.rename(columns={"text_":"text"})
dataframe = dataframe.drop(["category"], axis=1)

encoder = LabelEncoder()
encoder.fit(dataframe['label'])
dataframe['label_encoded'] = encoder.transform(dataframe['label'])

dataframe.head()
#df.label = df.label.apply(lambda v: 0 if v == "ham" else 1)

,rating,label,text,label_encoded
0,5.0,CG,"Love this! Well made, sturdy, and very comfor...",0
1,5.0,CG,"love it, a great upgrade from the original. I...",0
2,5.0,CG,This pillow saved my back. I love the look and...,0
3,1.0,CG,"Missing information on how to use it, but it i...",0
4,5.0,CG,Very nice set. Good quality. We have had the s...,0


In [ ]:
# why to use train test split and stratify: https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=420, shuffle=True, stratify=y)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional, GlobalAveragePooling1D

In [ ]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary

In [ ]:
#https://www.youtube.com/watch?v=Y_hzMnRXjhI&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=3
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train) # only fit to training data
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(x_train)

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(train_padded[0])
print(train_padded.shape)

In [ ]:
n_lstm = 20
drop_lstm =0.2

model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    #tf.keras.layers.GlobalAveragePooling1D(), 
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(64, activation='relu'),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_padded, y_train, epochs=10, validation_split=0.2)

In [ ]:
top_words = 5000
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

## deep learning

In [ ]:
%%time
def naive(n, smoothing):
    # https://kavita-ganesan.com/how-to-use-countvectorizer/
    cv = CountVectorizer(ngram_range=(n,n), stop_words="english", analyzer='char_wb') # ref scikit learn documentation
    train_features = cv.fit_transform(x_train)
    test_features = cv.transform(x_test)

    NB = MultinomialNB(alpha=smoothing)
    NB.fit(train_features, y_train)

    #pred = NB.predict(test_features)
    #score = metrics.accuracy_score(y_test, pred)
    #print(score)
    
    ## return here instead 
    print("accuracy is: ", NB.score(test_features, y_test))

    #cm = metrics.confusion_matrix(y_test, pred)
    #print(cm)

In [ ]:
%%time
ngrams = [1,2,3]
laplaces = [0.1, 1, 10]
#counts
for i in ngrams:
    for j in laplaces:
        print("ngrams: {} | laplace value: {}.".format(i, j))
        naive(i, j)

In [ ]:
# do fake and real reviews have the same legnth and polarity distribtuin. check word counts for each 
#do +ve and -ve reviews have the same length and dist
# wordclouds for fake and real 